In [1]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [48]:
out='../output/'
inb='../input/flows/'
pref='dam'
inp = inb + pref + '/'

In [49]:
dat=pd.DataFrame(columns=['date','flow','from','hour','to'])
onlyfiles = [f for f in listdir(inp ) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat['flow']=dat['flow'].astype(int)
dat

,date,flow,from,hour,to
0,2021-04-01,94,MOZN_EDM,0,ZIMA
1,2021-04-01,94,ZIMA,0,BOT
2,2021-04-01,47,MOZN_EDM,1,ZIMA
3,2021-04-01,47,ZIMA,1,BOT
4,2021-04-01,56,MOZN_EDM,2,ZIMA
...,...,...,...,...,...
214,2022-10-25,52,BOT,22,RSAN
215,2022-10-25,52,RSAN,22,ESW
216,2022-10-25,21,BOT,23,RSAN
217,2022-10-25,21,RSAN,23,ESW


In [50]:
dam = pd.read_csv(out + pref + '_hourly.csv')
dam=dam.set_index([pd.DatetimeIndex(dam['date']),'hour'])
dam=dam.drop(['date'],1)
dam

C:\Users\kabom\AppData\Local\Temp\ipykernel_33192\1849715270.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  dam=dam.drop(['date'],1)


price  volume  year  month  day  day_of_week   value  \
date       hour                                                         
2020-01-01 0      30.0    75.0  2020      1    1            2  2250.0   
           1      30.0    70.0  2020      1    1            2  2100.0   
           2       0.0     0.0  2020      1    1            2     0.0   
           3      30.0    65.0  2020      1    1            2  1950.0   
           4       0.0     0.0  2020      1    1            2     0.0   
...                ...     ...   ...    ...  ...          ...     ...   
2022-11-01 19      0.0     0.0  2022     11    1            1     0.0   
           20      0.0     0.0  2022     11    1            1     0.0   
           21      0.0     0.0  2022     11    1            1     0.0   
           22      0.0     0.0  2022     11    1            1     0.0   
           23      0.0     0.0  2022     11    1            1     0.0   

                      cat  
date       hour            
2020-01-01 0          off  
           1          off  
           2          off  
           3          off  
           4          off  
...                   ...  
2022-11-01 19     evening  
           20    standard  
           21    standard  
           22         off  
           23         off  

[22794 rows x 8 columns]

In [51]:
flow=dat
#flow['date']=pd.to_datetime(flow['flowdate'],format="%d/%m/%Y") #"%Y-%m-%d")#"%d/%m/%Y")
#flow=flow.drop(['flowdate','flowid','market'],axis=1)

flow=flow.set_index([pd.DatetimeIndex(flow['date'])])
flow['year']=flow.index.year
flow['month']=flow.index.month
flow['day']=flow.index.day
flow['day_of_week']=flow.index.dayofweek

flow=flow.set_index([pd.DatetimeIndex(flow['date']),'hour'])
flow['price']=dam['price']
flow['value']=flow['price']*flow['flow']
flow=flow.drop(['date'],axis=1)
flow=flow.dropna(0)
flow=flow.reset_index()

flow

C:\Users\kabom\AppData\Local\Temp\ipykernel_33192\2288514685.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  flow=flow.dropna(0)


,date,hour,flow,from,to,year,month,day,day_of_week,price,value
0,2021-04-01,0,94,MOZN_EDM,ZIMA,2021,4,1,3,30.0,2820.0
1,2021-04-01,0,94,ZIMA,BOT,2021,4,1,3,30.0,2820.0
2,2021-04-01,1,47,MOZN_EDM,ZIMA,2021,4,1,3,30.0,1410.0
3,2021-04-01,1,47,ZIMA,BOT,2021,4,1,3,30.0,1410.0
4,2021-04-01,2,56,MOZN_EDM,ZIMA,2021,4,1,3,30.0,1680.0
...,...,...,...,...,...,...,...,...,...,...,...
8651,2022-10-13,4,59,BOT,RSAN,2022,10,13,3,24.0,1416.0
8652,2022-10-13,4,59,RSAN,ESW,2022,10,13,3,24.0,1416.0
8653,2022-10-13,4,59,ZAMZ,ZIM,2022,10,13,3,24.0,1416.0
8654,2022-10-13,4,59,ZIM,ZIMA,2022,10,13,3,24.0,1416.0


In [52]:
f_to=flow[['date','hour','flow','value','price','to','year','month','day','day_of_week']].rename(columns={'to': 'node'})
f_from=flow[['date','hour','flow','value','price','from','year','month','day','day_of_week']].rename(columns={'from': 'node'})
f_from['flow']=-f_from['flow']
f_from['value']=-f_from['value']

flows=pd.concat([f_to, f_from])

flows

,date,hour,flow,value,price,node,year,month,day,day_of_week
0,2021-04-01,0,94,2820.0,30.0,ZIMA,2021,4,1,3
1,2021-04-01,0,94,2820.0,30.0,BOT,2021,4,1,3
2,2021-04-01,1,47,1410.0,30.0,ZIMA,2021,4,1,3
3,2021-04-01,1,47,1410.0,30.0,BOT,2021,4,1,3
4,2021-04-01,2,56,1680.0,30.0,ZIMA,2021,4,1,3
...,...,...,...,...,...,...,...,...,...,...
8651,2022-10-13,4,-59,-1416.0,24.0,BOT,2022,10,13,3
8652,2022-10-13,4,-59,-1416.0,24.0,RSAN,2022,10,13,3
8653,2022-10-13,4,-59,-1416.0,24.0,ZAMZ,2022,10,13,3
8654,2022-10-13,4,-59,-1416.0,24.0,ZIM,2022,10,13,3


In [53]:
flows.loc[flows['node']=='ZIMA','node']='ZIM'
flows.loc[flows['node']=='MOZN_EDM','node']='MOZ'
flows.loc[flows['node']=='MOZN_HCB','node']='HCB'
flows.loc[flows['node']=='MOZS','node']='MOZ'
flows.loc[flows['node']=='RSAS','node']='RSA'
flows.loc[flows['node']=='RSAN','node']='RSA'

In [54]:
flows=flows.reset_index().groupby(['date','hour','node','year','month','day','day_of_week']).sum().reset_index()
flows=flows.loc[flows['flow']!=0]
flows=flows.set_index(['date','hour'])
flows['cat']=dam['cat']
flows.reset_index()
flows

node  year  month  day  day_of_week  index  flow   value  \
date       hour                                                             
2021-04-01 0      BOT  2021      4    1            3      1    94  2820.0   
           0      MOZ  2021      4    1            3      0   -94 -2820.0   
           1      BOT  2021      4    1            3      3    47  1410.0   
           1      MOZ  2021      4    1            3      2   -47 -1410.0   
           2      BOT  2021      4    1            3      5    56  1680.0   
...               ...   ...    ...  ...          ...    ...   ...     ...   
2022-10-13 2     ZAMZ  2022     10   13            3   8643   -34  -816.0   
           3      ESW  2022     10   13            3   8647    36   864.0   
           3     ZAMZ  2022     10   13            3   8648   -36  -864.0   
           4      ESW  2022     10   13            3   8652    59  1416.0   
           4     ZAMZ  2022     10   13            3   8653   -59 -1416.0   

                 price  cat  
date       hour              
2021-04-01 0      30.0  off  
           0      30.0  off  
           1      30.0  off  
           1      30.0  off  
           2      30.0  off  
...                ...  ...  
2022-10-13 2      24.0  off  
           3      24.0  off  
           3      24.0  off  
           4      24.0  off  
           4      24.0  off  

[8929 rows x 10 columns]

In [55]:
flowsb=flows.copy()
flowsb.loc[flowsb['flow']<0,'side']='supply'
flowsb.loc[flowsb['flow']>0,'side']='consume'


flow_monthly_cat=flowsb.groupby(['year','month', 'node','cat','side']).sum()
flow_monthly_cat.rename(columns={'flow': 'flow_sum'}, inplace=True)
flow_monthly_cat['hour_count']=flowsb.groupby(['year','month', 'node','cat','side']).count()['flow']
flow_monthly_cat['price_weight_mean'] = flow_monthly_cat['value'] / flow_monthly_cat['flow_sum']
flow_monthly_cat['price_mean']=flowsb.groupby(['year','month', 'node','cat','side']).mean()['price']
flow_monthly_cat=flow_monthly_cat.drop(['day','day_of_week','index','price'],axis=1)
flow_monthly_cat=flow_monthly_cat.loc[flow_monthly_cat['flow_sum']!=0]
flow_monthly_cat['power']=flow_monthly_cat['flow_sum']/flow_monthly_cat['hour_count']
flow_monthly_cat

flow_sum      value  hour_count  \
year month node cat      side                                       
2021 4     BOT  evening  consume       137   13879.68           3   
                morning  consume      1178  105948.13          25   
                off      consume      2236   77258.43          51   
                standard consume      4816  319730.29          83   
           ESW  off      consume       186    6231.60           6   
...                                    ...        ...         ...   
2022 9     ZAMZ evening  supply        -40   -8400.00           2   
                off      supply       -165   -4823.00           6   
                standard supply       -369  -26800.75          13   
     10    ESW  off      consume       203    4872.00           5   
           ZAMZ off      supply       -203   -4872.00           5   

                                  price_weight_mean  price_mean      power  
year month node cat      side                                               
2021 4     BOT  evening  consume         101.311533  140.813333  45.666667  
                morning  consume          89.938990   89.113600  47.120000  
                off      consume          34.552071   35.630784  43.843137  
                standard consume          66.389180   75.195301  58.024096  
           ESW  off      consume          33.503226   33.493333  31.000000  
...                                             ...         ...        ...  
2022 9     ZAMZ evening  supply          210.000000  210.000000 -20.000000  
                off      supply           29.230303   29.000000 -27.500000  
                standard supply           72.630759   74.653846 -28.384615  
     10    ESW  off      consume          24.000000   24.000000  40.600000  
           ZAMZ off      supply           24.000000   24.000000 -40.600000  

[366 rows x 6 columns]

In [56]:
flow_monthly_cat.to_csv(out + pref + '_flow_monthly_cat.csv')